In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Input, Dense, Flatten, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

# --- Configuration ---
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 25
EPOCHS = 30 
DATASET_PATH = '/kaggle/input/plantvillage-dataset/color' 

MODEL_EXPORT_BASE_PATH = '/kaggle/working/models/plant_disease_detector'
MODEL_VERSION = 1
print(f"TensorFlow version: {tf.__version__}")
print(f"Is GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Cell 2: Data Loading and Preprocessing Function
def load_and_preprocess_data(dataset_path):
    # This will list folders like 'Apple___Apple_scab', 'Tomato___Early_blight'
    combined_disease_folders = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))])

    # Extract unique crop names from the folder names
    crop_types = sorted(list(set([folder.split('___')[0] for folder in combined_disease_folders])))
    crop_name_to_index = {name: i for i, name in enumerate(crop_types)}
    num_crop_types = len(crop_types)

    # The full disease labels will be the combined folder names themselves
    disease_labels = sorted(combined_disease_folders)
    full_disease_name_to_index = {name: i for i, name in enumerate(disease_labels)}
    num_disease_classes = len(disease_labels)

    # Collect all image paths and their corresponding labels
    all_image_paths = []
    all_labels_disease = []
    all_labels_crop_type = []

    print(f"Found {len(combined_disease_folders)} combined disease folders.")
    for combined_folder_name in combined_disease_folders:
        combined_folder_path = os.path.join(dataset_path, combined_folder_name)
        crop_name = combined_folder_name.split('___')[0]

        for img_name in os.listdir(combined_folder_path):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                all_image_paths.append(os.path.join(combined_folder_path, img_name))
                all_labels_disease.append(full_disease_name_to_index[combined_folder_name])
                all_labels_crop_type.append(crop_name_to_index[crop_name])

    print(f"Total images loaded: {len(all_image_paths)}")

    all_labels_disease = np.array(all_labels_disease)
    all_labels_crop_type = np.array(all_labels_crop_type)

    X_train_paths, X_val_paths, y_train_disease, y_val_disease, y_train_crop, y_val_crop = train_test_split(
        all_image_paths, all_labels_disease, all_labels_crop_type,
        test_size=0.2, random_state=42, stratify=all_labels_disease
    )

    def data_generator(image_paths, disease_labels_indices, crop_type_labels_indices, is_training=True):
        datagen = ImageDataGenerator(
            rescale=1./255, # Normalize pixel values to 0-1
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        ) if is_training else ImageDataGenerator(rescale=1./255)

        num_samples = len(image_paths)
        while True:
            indices = np.arange(num_samples)
            if is_training:
                np.random.shuffle(indices)

            for i in range(0, num_samples, BATCH_SIZE):
                batch_indices = indices[i:i + BATCH_SIZE]
                
                batch_image_paths = [image_paths[j] for j in batch_indices]
                batch_disease_labels = disease_labels_indices[batch_indices]
                batch_crop_type_labels = crop_type_labels_indices[batch_indices]

                images = []
                for img_path in batch_image_paths:
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
                    img = tf.keras.preprocessing.image.img_to_array(img)
                    images.append(img)
                
                images = np.array(images)
                
                if is_training:
                    # --- FIX IS HERE ---
                    # Replace .next() with .__next__() for newer TensorFlow versions
                    images = datagen.flow(images, batch_size=len(images), shuffle=False).__next__()
                else:
                    images = images / 255.0 # Manual rescale for validation without flow

                disease_one_hot = tf.keras.utils.to_categorical(batch_disease_labels, num_classes=len(disease_labels))
                crop_type_input_for_batch = np.array(batch_crop_type_labels)

                yield {"image_input": images, "crop_type_input": crop_type_input_for_batch}, disease_one_hot

    train_generator = data_generator(X_train_paths, y_train_disease, y_train_crop, is_training=True)
    val_generator = data_generator(X_val_paths, y_val_disease, y_val_crop, is_training=False)

    return train_generator, val_generator, \
           len(X_train_paths), len(X_val_paths), \
           num_crop_types, num_disease_classes, \
           crop_name_to_index, disease_labels

In [ ]:
# Cell 3: Build Model Function
def build_model(num_crop_types, num_disease_classes):
    image_input = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name='image_input')
    base_model = ResNet101(weights='imagenet', include_top=False, input_tensor=image_input)
    base_model.trainable = False # Freeze base model initially

    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    crop_type_input = Input(shape=(1,), name='crop_type_input', dtype='int32')
    y = tf.keras.layers.Embedding(input_dim=num_crop_types, output_dim=10)(crop_type_input)
    y = Flatten()(y)

    combined = concatenate([x, y])

    z = Dense(128, activation='relu')(combined)
    z = Dropout(0.3)(z)
    output = Dense(num_disease_classes, activation='softmax', name='predictions')(z)

    model = Model(inputs=[image_input, crop_type_input], outputs=output)

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model, base_model

In [ ]:
# Cell 4: Execute Data Loading and Get Mappings
print("Loading and preparing data...")
train_generator, val_generator, \
train_samples, val_samples, \
num_crop_types, num_disease_classes, \
crop_name_to_index, disease_labels = load_and_preprocess_data(DATASET_PATH)

print(f"\nDetected {num_crop_types} crop types and {num_disease_classes} total disease classes.")
print("Crop Name to Index Mapping:")
for crop_name, index in crop_name_to_index.items():
    print(f"  {crop_name}: {index}")
print("\nDisease Labels (full list):")
for i, label in enumerate(disease_labels):
    print(f"  {i}: {label}")

steps_per_epoch_train = train_samples // BATCH_SIZE
if train_samples % BATCH_SIZE != 0:
    steps_per_epoch_train += 1

steps_per_epoch_val = val_samples // BATCH_SIZE
if val_samples % BATCH_SIZE != 0:
    steps_per_epoch_val += 1

# Save mapping files
with open('crop_name_to_index.txt', 'w') as f:
    for name, idx in crop_name_to_index.items():
        f.write(f"{name}:{idx}\n")
with open('disease_labels.txt', 'w') as f:
    for label in disease_labels:
        f.write(f"{label}\n")
print("\nMapping files (crop_name_to_index.txt, disease_labels.txt) saved.")

In [ ]:
# Cell 5: Build and Summarize Model
print("\nBuilding model...")
model, base_model = build_model(num_crop_types, num_disease_classes)
model.summary()

In [ ]:
# Cell 6: Initial Training (Feature Extraction)
print("\nStarting Phase 1: Feature Extraction (Training top layers)...")

# Define Early Stopping callback
early_stopping_phase1 = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3, # Can be 3-5
    restore_best_weights=True,
    verbose=1
)

# Compile the model (if not already compiled or re-compiling for clarity)
# Ensure base_model.trainable is False before this compilation
# (It's already set to False in build_model function)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train Phase 1
history_phase1 = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=15, # Give it enough epochs to converge in Phase 1, but EarlyStopping will manage it
    validation_data=val_generator,
    validation_steps=steps_per_epoch_val,
    callbacks=[early_stopping_phase1]
)
print("Phase 1: Feature Extraction training complete.")

In [ ]:
# NEW Cell 7: Fine-Tuning Phase
print("\nStarting Phase 2: Fine-Tuning (Unfreezing and training parts of base model)...")

# Unfreeze the base model
base_model.trainable = True

# Optionally, unfreeze only a subset of layers (e.g., the last few convolutional blocks)
# This can prevent catastrophic forgetting of learned features.
# MobileNetV2 has ~155 layers including input/batchnorm.
# You might unfreeze layers from a certain index onwards, or a percentage from the end.
# For example, unfreeze the last ~20-30% of MobileNetV2 layers.
# Let's say we keep the first 100 layers frozen and unfreeze the rest.
# You might need to experiment with the 'unfreeze_from_layer' value.
UNFREEZE_FROM_LAYER = 250 # Adjust this value based on experimentation for ResNet101
for layer in base_model.layers[:UNFREEZE_FROM_LAYER]:
    layer.trainable = False
print(f"Number of trainable layers in base model after unfreezing: {len([layer for layer in base_model.trainable_variables])}")


# Compile the model with a much lower learning rate for fine-tuning
# This is CRUCIAL to prevent overwriting useful pre-trained features
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # A very small learning rate (e.g., 0.00001)
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define Early Stopping callback for fine-tuning
early_stopping_phase2 = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5, # Give it more patience for fine-tuning, as improvements can be subtle
    restore_best_weights=True,
    verbose=1
)

# Train Phase 2 (fine-tuning)
# You can set EPOCHS_FINE_TUNE higher, as EarlyStopping will manage it
EPOCHS_FINE_TUNE = 30 

history_phase2 = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=EPOCHS_FINE_TUNE,
    validation_data=val_generator,
    validation_steps=steps_per_epoch_val,
    callbacks=[early_stopping_phase2]
)
print("Phase 2: Fine-Tuning training complete.")

In [ ]:
# Cell 8: Visualize Training History
# Combine history objects for plotting
acc = history_phase1.history['accuracy'] + history_phase2.history['accuracy']
val_acc = history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy']

loss = history_phase1.history['loss'] + history_phase2.history['loss']
val_loss = history_phase1.history['val_loss'] + history_phase2.history['val_loss']

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy (Combined Phases)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss (Combined Phases)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Cell 9: Save Model for TensorFlow Serving
export_path = os.path.join(MODEL_EXPORT_BASE_PATH, str(MODEL_VERSION))

if os.path.exists(export_path):
    print(f"Removing existing model version at {export_path}")
    shutil.rmtree(export_path)

tf.saved_model.save(model, export_path)

print(f"Model saved to: {export_path}")

# To easily download the model folder and mapping files from Kaggle's "Output" tab
# You need to ensure they are written to /kaggle/working/
# The .txt files are already saved there.
# We can zip the model directory for easier download.

# Zip the model folder
model_zip_path = f"/kaggle/working/plant_disease_detector_model_v{MODEL_VERSION}.zip"
shutil.make_archive(
    base_name=os.path.join('/kaggle/working', f"plant_disease_detector_model_v{MODEL_VERSION}"),
    format='zip',
    root_dir='/kaggle/working/models', # Base directory to start archiving from
    base_dir='plant_disease_detector'  # The specific directory inside root_dir to archive
)
print(f"\nModel zipped to: {model_zip_path}")
print("Now, 'Save Version' (Commit) your notebook.")
print("After the commit run completes, go to the 'Output' tab of that version.")
print("You will find 'plant_disease_detector_model_v1.zip', 'crop_name_to_index.txt', and 'disease_labels.txt' available for download.")